#Boiler Plate

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import gc
import os

In [ ]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions: 
# tf.compat.v1.set_random_seed(seed_value)

# 5. Configure a new global `tensorflow` session
from keras import backend as K
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)

#Load Data

All Data can be found at this anonymous repo https://anonymous.4open.science/r/CHILData-4164/

In [ ]:
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/trainingPerfusionLargeCloseClin.pkl", "rb")
trainingPerfusion = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/validationPerfusionLargeCloseClin.pkl", "rb")
validationPerfusion = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/testPerfusionLargeCloseClin.pkl", "rb")
testPerfusion = pickle.load(o)

o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/trainingClinicalCloseClin.pkl", "rb")
trainingClinical = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/validationClinicalCloseClin.pkl", "rb")
validationClinical = pickle.load(o)
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/testClinicalCloseClin.pkl", "rb")
testClinical = pickle.load(o)


In [ ]:
labelTraining = np.append(np.ones(54), np.zeros(54))
labelValTest = np.append(np.ones(18), np.zeros(18))

# Train all Models

In [ ]:
from keras import layers, models, optimizers, metrics
from keras.callbacks import EarlyStopping, ModelCheckpoint
!pip install keras-tuner
import kerastuner as kt
from kerastuner import HyperParameters as hp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 31.3 MB/s eta 0:00:00


<ipython-input-6-44a22687b9c5>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
timeLen = trainingPerfusion[0,0].shape[1]
varLen = trainingPerfusion[0,0].shape[2]

def TransfomerEncoder(input, sizeOne, dropoutOne):

    # #Conv
    conv = layers.Conv1D(sizeOne, 1, padding = "same", activation="tanh")(input)
    #convolution Layer
    lstm = layers.LSTM(sizeOne, dropout = dropoutOne, return_sequences=True)(conv) #res
    add = layers.Add()([conv, lstm])
    lstm = layers.LSTM(sizeOne, dropout = dropoutOne)(add)
    lstm = layers.LayerNormalization()(lstm)
    
    return lstm

def clinicalClass(input, sizeOne, dropoutOne, sizeTwo, dropoutTwo):
    
    classif = layers.Dense(sizeOne, activation = "selu")(input)
    dropout = layers.Dropout(dropoutOne)(classif)
    classif = layers.Dense(input.shape[1], activation = "selu")(dropout)
    dropout = layers.Dropout(dropoutOne)(classif)
    finalLayer = layers.Dense(sizeTwo, activation = "selu")(dropout)
    finalDropout = layers.Dropout(dropoutTwo)(finalLayer)
    
    return finalDropout




In [ ]:
def make_model():
  input_layerPerf = layers.Input(shape = (timeLen,varLen,), name="PerfusionData" ) 
  print(input_layerPerf.shape)
  input_layerClin = layers.Input(shape = ([trainingClinical.shape[3],]), name="ClinData" ) 
  print(input_layerClin.shape)
  
  x = input_layerPerf
  x = TransfomerEncoder(x, 1024, 0.3)
  y = input_layerClin
  y = clinicalClass(y,33,0,25,0.2)

  concat = layers.Concatenate()([x,y])
  output_layer = layers.Dense(1, activation="sigmoid", name="Prediction")(concat)

  model = models.Model(inputs=[input_layerPerf,input_layerClin ], outputs=output_layer)

  ##Model Creation
  opt = tf.keras.optimizers.Adam(learning_rate=0.00001)
  model.compile(loss='binary_crossentropy', optimizer=opt, metrics=
                ["acc",
                 tf.keras.metrics.AUC(name='auc_min', summation_method = "minoring"),
                 tf.keras.metrics.AUC(name='auc_max', summation_method = "majoring")],)

  return model

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=0,  patience=20, min_delta = 0.00001)
predictions = np.zeros([12,5,36])


for x in range(12):
  for j in range(5):
    mc = ModelCheckpoint("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/LSTMModelsLarge/CloseClinK{}Fold{}.h5".format(x, j), monitor='val_loss', mode='min', verbose=0, save_best_only=True)
    model = make_model()
    model.fit(
        {"PerfusionData": trainingPerfusion[x,j],"ClinData": trainingClinical[x,j]},
        {"Prediction": labelTraining},
        epochs=10000,   
        # batch_size = trainingPerfusion.shape[2],                                                     
        validation_data = ([validationPerfusion[x,j],validationClinical[x,j]], labelValTest),
        callbacks=[es, mc],
        verbose=1,
    )
    model.load_weights("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/LSTMModelsLarge/CloseClinK{}Fold{}.h5".format(x, j))
    yPred = model.predict({"ClinData": testClinical[x,j],"PerfusionData": testPerfusion[x,j]}) 
    predictions[x,j] = yPred.reshape(-1)


(None, 2054, 8)
(None, 3)
Epoch 1/10000
4/4 [==============================] - 17s 895ms/step - loss: 0.7592 - acc: 0.4815 - auc_min: 0.4866 - auc_max: 0.5038 - val_loss: 0.5781 - val_acc: 0.6667 - val_auc_min: 0.9815 - val_auc_max: 0.9815
Epoch 2/10000
4/4 [==============================] - 2s 502ms/step - loss: 0.5447 - acc: 0.7685 - auc_min: 0.8632 - auc_max: 0.8687 - val_loss: 0.4520 - val_acc: 0.8056 - val_auc_min: 0.9815 - val_auc_max: 0.9815
Epoch 3/10000
4/4 [==============================] - 2s 502ms/step - loss: 0.4620 - acc: 0.7963 - auc_min: 0.9091 - auc_max: 0.9132 - val_loss: 0.3804 - val_acc: 0.8889 - val_auc_min: 0.9815 - val_auc_max: 0.9815
Epoch 4/10000
4/4 [==============================] - 2s 507ms/step - loss: 0.3994 - acc: 0.8426 - auc_min: 0.9201 - auc_max: 0.9256 - val_loss: 0.3703 - val_acc: 0.8333 - val_auc_min: 0.9815 - val_auc_max: 0.9815
Epoch 5/10000
4/4 [==============================] - 2s 504ms/step - loss: 0.3595 - acc: 0.8796 - auc_min: 0.9242 - auc_m

2/2 [==============================] - 1s 101ms/step
(None, 2054, 8)
(None, 3)
Epoch 1/10000
4/4 [==============================] - 6s 794ms/step - loss: 0.6652 - acc: 0.4907 - auc_min: 0.6235 - auc_max: 0.6351 - val_loss: 0.3619 - val_acc: 1.0000 - val_auc_min: 1.0000 - val_auc_max: 1.0000
Epoch 2/10000
4/4 [==============================] - 2s 498ms/step - loss: 0.5220 - acc: 0.8056 - auc_min: 0.8755 - auc_max: 0.8817 - val_loss: 0.2295 - val_acc: 1.0000 - val_auc_min: 1.0000 - val_auc_max: 1.0000
Epoch 3/10000
4/4 [==============================] - 2s 479ms/step - loss: 0.4652 - acc: 0.7685 - auc_min: 0.8573 - auc_max: 0.8611 - val_loss: 0.1786 - val_acc: 0.9444 - val_auc_min: 1.0000 - val_auc_max: 1.0000
Epoch 4/10000
4/4 [==============================] - 2s 500ms/step - loss: 0.4058 - acc: 0.7778 - auc_min: 0.9005 - auc_max: 0.9029 - val_loss: 0.1331 - val_acc: 0.9444 - val_auc_min: 1.0000 - val_auc_max: 1.0000
Epoch 5/10000
4/4 [==============================] - 1s 322ms/step - 

2/2 [==============================] - 1s 95ms/step
(None, 2054, 8)
(None, 3)
Epoch 1/10000
4/4 [==============================] - 6s 792ms/step - loss: 0.7796 - acc: 0.4537 - auc_min: 0.4558 - auc_max: 0.4630 - val_loss: 0.5432 - val_acc: 0.8056 - val_auc_min: 0.9444 - val_auc_max: 0.9444
Epoch 2/10000
4/4 [==============================] - 2s 501ms/step - loss: 0.5774 - acc: 0.7407 - auc_min: 0.7877 - auc_max: 0.7925 - val_loss: 0.4290 - val_acc: 0.9722 - val_auc_min: 1.0000 - val_auc_max: 1.0000
Epoch 3/10000
4/4 [==============================] - 2s 482ms/step - loss: 0.5044 - acc: 0.7500 - auc_min: 0.8361 - auc_max: 0.8378 - val_loss: 0.3514 - val_acc: 0.8889 - val_auc_min: 1.0000 - val_auc_max: 1.0000
Epoch 4/10000
4/4 [==============================] - 2s 498ms/step - loss: 0.4504 - acc: 0.7222 - auc_min: 0.8618 - auc_max: 0.8666 - val_loss: 0.3104 - val_acc: 0.9167 - val_auc_min: 1.0000 - val_auc_max: 1.0000
Epoch 5/10000
4/4 [==============================] - 2s 487ms/step - l

In [ ]:
f = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/predictionsLargeCloseClin.pkl","wb")
pickle.dump(predictions,f)
f.close()

#Measure Performance

In [ ]:
o = open("/content/drive/My Drive/ECMOProj/GaussianProcessForArtData/predictionsLargeCloseClin.pkl", "rb")
predictions = pickle.load(o)

In [ ]:
predictions.shape

(12, 5, 36)

In [ ]:
from sklearn.metrics import roc_auc_score
def bootstrapAUC(true, preds, bootstrap = 5000):
  AUCTotal = np.zeros([0])
  if((true.sum()<1) or (true.shape[0]-true.sum() <1)):
    print("Less than 1 true or false")
    return AUCTotal
  for x in range(bootstrap):
    np.random.seed(x)
    randInt = np.random.randint(0,true.shape[0], true.shape[0])
    trueSub = true[randInt]
    predSub = preds[randInt]

    while((trueSub.sum()<1) or (trueSub.shape[0]-trueSub.sum() <1)):
      randInt = np.random.randint(0,true.shape[0], true.shape[0])
      trueSub = true[randInt]
      predSub = preds[randInt]

    AUC = roc_auc_score(trueSub, predSub)
    AUCTotal = np.append(AUCTotal, AUC)

  return AUCTotal

In [ ]:
AllLabel = np.tile(np.append(np.ones([18]), np.zeros([18])), 60)
bootStrappedAUC =  bootstrapAUC(AllLabel, predictions.flatten())

In [ ]:
bootStrappedAUC.mean()
#0.7640249822669571

0.7640249822669571

In [ ]:
print("[{:.4f} - {:.4f}]".format(np.percentile(bootStrappedAUC, 2.5),np.percentile(bootStrappedAUC, 97.5)))
#[0.7424 - 0.7849]

[0.7424 - 0.7849]


In [ ]:
#